Привет еще раз, меня зовут Люман Аблаев. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ..
</div>
<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

<font color='orange' style='font-size:24px; font-weight:bold'>Полезные ссылки:</font>
* Полезная лекция про временные ряды: https://www.youtube.com/watch?v=u433nrxdf5k .

<font color='orange' style='font-size:24px; font-weight:bold'>Общее впечатление</font>
* Большое спасибо за проделанную работу. Видно, что приложено много усилий.
- Приятное оформление выводов и структурированность кода в целом!  Приятно проверять такие работы.
* Над этим проектом нужно будет еще немного поработать. Однако, изменения не должны занять много времени.
* В работе я оставил несколько советов. Буду рад, если ты учтешь их.
- Жду твой обновленный проект =)


# Taxi demand predicting 🚕

Компания «Чётенькое такси» собрала исторические данные о заказах такси в аэропортах. Чтобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час. Требуется построить модель для такого предсказания.

**Требование по качеству:** `RMSE <= 48` на тестовой выборке

**Содержание**<a id='toc0_'></a>    
1. [Подготовка     ](#toc1_)    
1.1. [Библиотеки     ](#toc1_1_)    
1.2. [Конфигурация     ](#toc1_2_)    
1.3. [Сервисные функции     ](#toc1_3_)    
2. [Данные     ](#toc2_)    
2.1. [Загрузка     ](#toc2_1_)    
2.2. [Проверка типов данных     ](#toc2_2_)    
2.3. [Проверка пропусков     ](#toc2_3_)    
2.4. [Ресемплирование     ](#toc2_4_)    
2.5. [Изучение признаков   ](#toc2_5_)    
2.6. [Вывод по датасету  ](#toc2_6_)    
3. [Модели     ](#toc3_)    
3.1. [Подготовка  ](#toc3_1_)    
3.2. [`LinearRegression`  ](#toc3_2_)    
3.3. [`RandomForestRegressor`  ](#toc3_3_)    
3.4. [`LGBMRegressor`  ](#toc3_4_)    
3.5. [`CatBoostRegressor`  ](#toc3_5_)    
4. [Результаты  ](#toc4_)    
4.1. [Сравнение моделей ](#toc4_1_)    
4.2. [Важность признаков](#toc4_2_)    
5. [Общий вывод     ](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>Подготовка      [&#8593;](#toc0_)

### 1.1. <a id='toc1_1_'></a>Библиотеки      [&#8593;](#toc0_)

In [ ]:
from  IPython.display import clear_output

In [ ]:
%pip install -q catboost==1.2.7
%pip install -q hyperopt==0.2.7
%pip install -q lightgbm==4.5.0
%pip install -q matplotlib==3.8.4
%pip install -q numpy==1.26.4
%pip install -q pandas==2.2.3
%pip install -q phik==0.12.4
%pip install -q prettytable==3.12.0
%pip install -q scikit-learn==1.5.2
%pip install -q seaborn==0.13.2
%pip install -q shap==0.46.0
%pip install -q statsmodels==0.14.4
%pip install -q termcolor==2.5.0

clear_output()

In [ ]:
from os.path import exists

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap

# Hyperparams tuning
from hyperopt import (hp,
                      fmin,
                      tpe,
                      Trials,
                      STATUS_OK)

# Models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from prettytable import PrettyTable
from termcolor import colored
from statsmodels.tsa.seasonal import seasonal_decompose


<div class="alert alert-block alert-success">
<b>Успех:</b> Импорты  как всегда на месте
</div>

### 1.2. <a id='toc1_2_'></a>Конфигурация      [&#8593;](#toc0_)

In [ ]:
RANDOM_STATE = 27
TEST_SIZE = 0.1

### 1.3. <a id='toc1_3_'></a>Сервисные функции      [&#8593;](#toc0_)

In [ ]:
def get_dataframe(paths: list[str], **kwargs) -> pd.DataFrame:
    for _path in paths:
        if not exists(_path) and not _path.startswith('http'):
            continue

        try:
            df = pd.read_csv(_path, **kwargs)
        except:
            continue

        if df is None:
            continue

        return df

    raise FileNotFoundError('No paths are valid for correct csv file.')

In [ ]:
def check_nans(df: pd.DataFrame) -> None:
    if df.isna().sum().sum() == 0:
        print(colored('Полных дубликатов не обнаружено.', 'green'))
        return

    table = PrettyTable()
    table.field_names = ['Feature', 'Missing values count']

    missing_info = df.isna().sum().sort_values()
    cols = missing_info.index.to_list()
    for col in cols:
        count = missing_info[col]
        color = 'green' if count == 0 else 'red'
        s = f'{count} ({count / len(df):.2%})'
        table.add_row([col, colored(s, color)])

    print(table)

In [ ]:
def draw_plots_about_column(df: pd.DataFrame,
                            column: str,
                            column_description: str = None,
                            bins: int = 20
                            ) -> None:
    if column_description is None:
        column_description = column

    _, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 10))

    sns.violinplot(df, x=column, ax=ax1, orient='h')
    ax1.set_xlabel(column_description)
    ax1.grid()

    sns.boxplot(df, x=column, ax=ax2)
    ax2.set_xlabel(column_description)
    ax2.grid()

    sns.histplot(df, x=column, bins=bins, ax=ax3)
    ax3.set_xlabel(column_description)
    ax3.grid()
    ax3.set_ylabel('Количество')

    ax4.set_visible(False)

    plt.suptitle(f'Информация о колонке `{column_description}`', fontsize=18)
    plt.tight_layout()

    plt.show()


<div class="alert alert-block alert-success">
<b>Успех:</b> Хорошие помощники
</div>

## 2. <a id='toc2_'></a>Данные      [&#8593;](#toc0_)

### 2.1. <a id='toc2_1_'></a>Загрузка      [&#8593;](#toc0_)

In [ ]:
df = get_dataframe([
        './taxi.csv',
        '/datasets/taxi.csv',
        'https://code.s3.yandex.net/datasets/taxi.csv'
    ], parse_dates=['datetime'], index_col='datetime'
).sort_index()


<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что использолованы параметры index_col и parse_dates
</div>


In [ ]:
df.head()

### 2.2. <a id='toc2_2_'></a>Проверка типов данных      [&#8593;](#toc0_)

In [ ]:
df.info()

In [ ]:
df.index.is_monotonic_increasing

### 2.3. <a id='toc2_3_'></a>Проверка пропусков      [&#8593;](#toc0_)

In [ ]:
check_nans(df)

### 2.4. <a id='toc2_4_'></a>Ресемплирование      [&#8593;](#toc0_)

Предсказание нужно сделать на следующий час, поэтому ресемпилируем по одному часу.

In [ ]:
df.head()

In [ ]:
df = df.resample('1h').sum()

In [ ]:
df.head()


<div class="alert alert-block alert-success">
<b>Успех:</b> Первичный осмотр проведен, ресемпилнг был сделан верно.
</div>


### 2.5. <a id='toc2_5_'></a>Изучение признаков    [&#8593;](#toc0_)

In [ ]:
df['num_orders'].describe()

In [ ]:
draw_plots_about_column(df, 'num_orders')

In [ ]:
_ ,ax = plt.subplots(figsize=(20, 7))
df.plot(ax=ax);

In [ ]:
df_for_analysis = df.copy()

In [ ]:
_ ,ax = plt.subplots(figsize=(20, 7))
df_for_analysis['rolling_mean'] = df_for_analysis['num_orders'].shift().rolling(7).mean()
df_for_analysis.plot(ax=ax)
plt.title('Изменение кол-ва заказов по времени')
plt.xlabel('Время')
plt.ylabel('Кол-во заказов, шт.')
plt.show()

In [ ]:
decomposed = seasonal_decompose(df)

In [ ]:
plt.figure(figsize=(10, 15))
plt.subplot(311)
decomposed.trend.plot(ax=plt.gca())
plt.title('Trend')

plt.subplot(312)
decomposed.seasonal.plot(ax=plt.gca())
plt.title('Seasonality')

plt.subplot(313)
decomposed.resid.plot(ax=plt.gca())
plt.title('Residuals')

plt.tight_layout()
plt.show()

### 2.6. <a id='toc2_6_'></a>Вывод по датасету   [&#8593;](#toc0_)


Перед нами датасет с информацией о заказах такси. Изначально данные даны по каждым 10 минутам. Однако в данном проекте нас интересует более крупный масштаб, поэтому данные были ресемплированы по часу.

Все типы данных корректны, пропусков нет.

Целевой признак (кол-во заказов) без аномалий, все значения вполне реальны.

Анализ трендов выявил рост кол-ва заказов с каждым месяцем,сезонность не показательна.

<div class="alert alert-block alert-danger">
  
<b>Ошибка:</b> Хотелось бы услышать, побольше слов по аналитике
    
По тренду: он постоянный, с ускорением? Растет число заказов по какой месяц? а затем как обстоят дела?
    
Можно посмотреть в другом масштабе на данные: есть ли суточная сезонность? в какие часы больше и меньше заказов? Можно взять данные за несколько суток.
    
Важно также изучить недельную сезонность. Удобнее всего ее увидеть, если сделать дополнительное ресемплирование (сохраняем в другую переменную, так как дальше в проекте будем работать с ресемплированием по часам и важно его не потерять) по дням. И также берем несколько недель.


По «остаткам» или шуму. Нужно посмотреть, на всем ли протяжении доля остатков одинаковая? Если если «остатков» к концу графика больше, значит в тестовую выборку попадет чуть больше данных, которые сложно объяснить и на которых модели будет сложнее предсказывать. Поэтому относительно большая (по сравнению с другими задачами машинного обучения) разница между метриками при обучении и исследовании, и между тестовой выборкой, не обязательно будет объясняться только переобучением модели.

       
</div>

<div class="alert alert-block alert-info">
    
<b>Совет:</b> Дополнительно  полезно оценивать стационарность ряда с помощью теста Дики-Фуллера: в реальных задачах с временными рядами, в особенности - с финансовыми данными или биржевыми котировками, график не всегда поможет достоверно оценить стационарность ряда, поэтому для оценки стационарности ряда лучше использовать тест Дики — Фуллера (подробнее о нём можно глянуть тут: https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.adfuller.html, а вот тут можно глянуть пример реализации:  https://habr.com/ru/articles/540868/) - он даст наиболее точный результат. Результат нужно проверить на уровнях значимости 1% и 5%, чтобы быть уверенным в выводах. Не забудь при применении теста сформулировать гипотезы - нулевая формулируется как гипотеза о нестацинарности ряда, а альтернативная - как гипотеза о стационарности ряда. Также при проведении теста не забывай, что мы сравниваем `p-value` с уровнем значимости `alpha`.
</div>


## 3. <a id='toc3_'></a>Модели      [&#8593;](#toc0_)

### 3.1. <a id='toc3_1_'></a>Подготовка   [&#8593;](#toc0_)

In [ ]:
def generate_features(df: pd.DataFrame, max_lag: int, rolling_mean_size: int) -> pd.DataFrame:
        df['month'] = df.index.month
        df['day'] = df.index.day
        df['dayofweek'] = df.index.dayofweek

        for lag in range(1, max_lag + 1):
            df['lag_{}'.format(lag)] = df['num_orders'].shift(lag)

        df['rolling_mean'] = df['num_orders'].shift().rolling(rolling_mean_size).mean()

        return df.dropna()


<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Не все выбранные признаки нам подходят:
    

Фактор месяц:  у нас в истории всего один март, май.... Да и горизонт прогноза - часы.


А вот фактор час - хорошо было бы включить.

Наши календарные признаки подчеркивают сезонность. Относительно сезонности признаки идут на один уровень ниже. Если у нас суточная сезонность, то логично создавать признак часа (час одного дня похож на тот же час другого), если недельная - день недели.
</div>	

In [ ]:
def get_objective(estimator_class, df: pd.DataFrame, fixed_params: dict):
    def objective(params: dict) -> dict:
        # Некоторые параметры могут быть только целочисленные, приводим их к нужному типу
        for key, value in params.items():
            if isinstance(value, float) and value % 1 == 0:
                params[key] = int(value)

        if 'max_lag' not in params or 'rolling_mean_size' not in params:
            raise ValueError()

        df_with_features = generate_features(df, params['max_lag'], params['rolling_mean_size'])
        X = df_with_features.drop(columns='num_orders')
        y = df_with_features['num_orders']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, shuffle=False)

        params_copy = dict(params)
        del params['max_lag']
        del params['rolling_mean_size']
        estimator = estimator_class(**fixed_params, **params)

        estimator.fit(X_train, y_train)
        y_pred = estimator.predict(X_test)
        rmse = root_mean_squared_error(y_test, y_pred)

        return {
            'loss': rmse,
            'params': params_copy,
            'status': STATUS_OK
        }

    return objective



<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Хорошо устроена логика работы с функциями, но не хорошо, чтобы настраиваешь модель, используя тестовую выборку. Постарайся выделить еще валидационную либо примени кросс-валидацию на трейне.
</div>


In [ ]:
def get_model_results(estimator_class,
                      param_space: dict,
                      df: pd.DataFrame,
                      fixed_params: dict,
                      max_evals: int = 100
                      ) -> tuple[dict, float]:
    objective = get_objective(estimator_class, df, fixed_params)
    trials = Trials()

    fmin(
        fn=objective,
        space=param_space,
        algo=tpe.suggest,
        max_evals=max_evals,
        trials=trials,
        rstate=np.random.default_rng(RANDOM_STATE),
        show_progressbar=True
    )

    clear_output()

    best_params = trials.best_trial['result']['params']
    best_score = abs(trials.best_trial['result']['loss'])
    print(f'Finish with best RMSE = {best_score:.2f}')

    return best_params, best_score

### 3.2. <a id='toc3_2_'></a>`LinearRegression`   [&#8593;](#toc0_)

In [ ]:
param_space = {
    'max_lag': hp.quniform('max_lag', 1, 15, 1),
    'rolling_mean_size': hp.quniform('rolling_mean_size', 1, 15, 1),
}

fixed_params = {
    'n_jobs': -1
}

In [ ]:
best_params_lr, best_score_lr = get_model_results(LinearRegression, param_space, df, fixed_params, max_evals=225)

### 3.3. <a id='toc3_3_'></a>`RandomForestRegressor`   [&#8593;](#toc0_)

In [ ]:
param_space = {
    'max_lag': hp.quniform('max_lag', 1, 15, 1),
    'rolling_mean_size': hp.quniform('rolling_mean_size', 1, 15, 1),

    'n_estimators': hp.quniform('n_estimators', 10, 200, 1),         # Количество деревьев в лесу
    'max_depth': hp.quniform('max_depth', 1, 30, 1),                 # Максимальная глубина дерева
    'min_samples_split': hp.uniform('min_samples_split', 0.1, 1.0),  # Минимальное количество образцов для разделения узла
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0.1, 0.5),    # Минимальное количество образцов в листе
    'max_features': hp.choice('max_features', ['sqrt', 'log2']),     # Количество признаков для рассмотрения при поиске лучшего разделения
    'bootstrap': hp.choice('bootstrap', [True, False]),              # Использование bootstrap выборок при построении деревьев
}

fixed_params = {
    'n_jobs': -1,
    'random_state': RANDOM_STATE
}

In [ ]:
best_params_rfr, best_score_rfr = get_model_results(RandomForestRegressor, param_space, df, fixed_params, max_evals=1000)

### 3.4. <a id='toc3_4_'></a>`LGBMRegressor`   [&#8593;](#toc0_)

In [ ]:
param_space = {
    'max_lag': hp.quniform('max_lag', 1, 15, 1),
    'rolling_mean_size': hp.quniform('rolling_mean_size', 1, 15, 1),

    'num_leaves': hp.quniform('num_leaves', 20, 150, 1),
    'max_depth': hp.quniform('max_depth', 3, 12, 1),
    'learning_rate': hp.loguniform('learning_rate', -5, 0),
    'n_estimators': hp.quniform('n_estimators', 100, 1000, 10),
    'min_child_samples': hp.quniform('min_child_samples', 5, 100, 1),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, 1),
    'reg_lambda': hp.loguniform('reg_lambda', -5, 1)
}

fixed_params = {
    'n_jobs': -1,
    'random_state': RANDOM_STATE,
    'verbose': -1
}

In [ ]:
best_params_lgbm, best_score_lgbm = get_model_results(LGBMRegressor, param_space, df, fixed_params, max_evals=200)

### 3.5. <a id='toc3_5_'></a>`CatBoostRegressor`   [&#8593;](#toc0_)

In [ ]:
param_space = {
    'max_lag': hp.quniform('max_lag', 1, 15, 1),
    'rolling_mean_size': hp.quniform('rolling_mean_size', 1, 15, 1),

    'iterations': hp.choice('iterations', range(100, 1001)),          # Количество деревьев (итераций)
    'learning_rate': hp.loguniform('learning_rate', -5, 0),           # Скорость обучения (логарифмический масштаб)
    'depth': hp.choice('depth', range(4, 11)),                        # Глубина деревьев
    'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 10),                  # Коэффициент L2-регуляризации
    'random_strength': hp.uniform('random_strength', 0, 10),          # Сила случайности при выборе разбиений
    'bagging_temperature': hp.uniform('bagging_temperature', 0, 1),   # Температура бэггинга
    'border_count': hp.choice('border_count', [32, 64, 128, 254]),    # Количество границ для числовых признаков
    'min_data_in_leaf': hp.choice('min_data_in_leaf', range(1, 21)),  # Минимальное количество данных в листе
    'max_leaves': hp.choice('max_leaves', range(10, 101)),            # Максимальное количество листьев (для 'Lossguide')
}

fixed_params = {
    'thread_count': -1,
    'random_state': RANDOM_STATE,
    'silent': True,
    'grow_policy': 'Lossguide'
}

In [ ]:
best_params_cbr, best_score_cbr = get_model_results(CatBoostRegressor, param_space, df, fixed_params, max_evals=200)

## 4. <a id='toc4_'></a>Результаты   [&#8593;](#toc0_)

### 4.1. <a id='toc4_1_'></a>Сравнение моделей  [&#8593;](#toc0_)

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

pd.DataFrame(
    data=[best_score_rfr, best_score_cbr, best_score_lgbm, best_score_lr],
    index=['RandomForestClassifier', 'CatBoostClassifier', 'LGBMClassifier', 'LinearRegression'],
    columns=['rmse']
).sort_values('rmse', ascending=True)

2 модели из 4 не прошли по целевой метрике, а вот другая половина показала удовлетворительный результат (хоть метрика и находится на пороге).


<div class="alert alert-block alert-success">
<b>Успех:</b> А так хорошо проделан шаг, радует количество рассмотренных моделей
</div>




<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Выбери здесь 1 лучшую модель и оцени только ее на тестовой выборке
</div>

### 4.2. <a id='toc4_2_'></a>Важность признаков [&#8593;](#toc0_)

In [ ]:
max_lag = best_params_cbr['max_lag']
rolling_mean_size = best_params_cbr['rolling_mean_size']

del best_params_cbr['max_lag']
del best_params_cbr['rolling_mean_size']

In [ ]:
df_for_shap = generate_features(df, max_lag, rolling_mean_size)
X_for_shap = df_for_shap.drop(columns='num_orders')
y_for_shap = df_for_shap['num_orders']

In [ ]:
best_model = CatBoostRegressor(grow_policy='Lossguide', thread_count=-1, random_state=RANDOM_STATE, silent=True, **best_params_cbr)
best_model.fit(X_for_shap, y_for_shap)
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_for_shap)

shap.summary_plot(shap_values, X_for_shap, color_bar_label='Признаки', show=False, plot_type='bar', show_values_in_legend=True, max_display=100)
plt.title('SHAP важность признаков')
plt.xlabel('SHAP значение')
plt.ylabel('Признаки', fontsize=13)

plt.tight_layout()
plt.show()


<div class="alert alert-block alert-success">
<b>Успех:</b> Лайк за важность факторов - отличный для аналитики инструмент
</div>


<div class="alert alert-info">
<b>Совет:</b>

Здесь также было бы очень полезно визуализировать предсказания твоей лучшей модели и сравнить их с правильными ответами в формате линейного графика - так мы сможем более наглядно понять, в каких местах твоя модель делает хорошие предсказания, а в каких ошибается.
        
По графику обязательно нужно сделать вывод: стоит отметить, в каких участках ряда модель сильнее ошибается, а какие участки ряда предсказывает хорошо.

</div>

<div class=

## 5. <a id='toc5_'></a>Общий вывод      [&#8593;](#toc0_)

В рамках данного проекта мы работали с датасетом о заказах такси с шагом 10 минут. Датасет не содержит пропусков. Индекс (дата-время заказа) монотонный. Целевой признак - количество заказов (`num_orders`) содержит адекватную информацию (без аномалий).

Анализ трендов выявил рост с течением времени. Сезонность не показала ничего.

Было протестировано 4 различные модели с перебором гиперпараметров. Помимо гиперпараметров модели перебирались также параметры генерация новых признаков (максимальный лаг и размер окна для расчёта скользящего среднего). В результате 2 модели (*LinearRegression* и *RandomForestRegressor*) показали метрику ниже допустимой. А вот 2 другие - *LGBMRegressor* и *CatBoostRegressor* показали удовлетворительные результаты (хоть метрика и была на грани). С минимальным отрывом победил *CatBoostRegressor*, поэтому именно он рекомендуется для дальнейшей работы.

Анализ важности признаков выявил, что 2 ключевыми признаками являются предыдущее значение (лаг 1) и месяц заказа.